## gat

In [2]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import PPI
from torch_geometric.data import DataLoader
from torch_geometric.nn import GATConv
from sklearn import metrics
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '2,3'

path = osp.join('./', 'data', 'PPI')
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='test')
test_dataset = PPI(path, split='test')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(train_dataset.num_features, 256, heads=4)
        self.lin1 = torch.nn.Linear(train_dataset.num_features, 4 * 256)
        self.conv2 = GATConv(4 * 256, 256, heads=4)
        self.lin2 = torch.nn.Linear(4 * 256, 4 * 256)
        self.conv3 = GATConv(
            4 * 256, train_dataset.num_classes, heads=6, concat=False)
        self.lin3 = torch.nn.Linear(4 * 256, train_dataset.num_classes)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index) + self.lin1(x))
        x = F.elu(self.conv2(x, edge_index) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x


device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        num_graphs = data.num_graphs
        data.batch = None
        data = data.to(device)
        optimizer.zero_grad()
        loss = loss_op(model(data.x, data.edge_index), data.y)
        total_loss += loss.item() * num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)


def test(loader):
    model.eval()

    total_micro_f1 = 0
    for data in loader:
        with torch.no_grad():
            out = model(data.x.to(device), data.edge_index.to(device))
        pred = (out > 0).float().cpu()
        micro_f1 = metrics.f1_score(data.y, pred, average='micro')
        total_micro_f1 += micro_f1 * data.num_graphs
    return total_micro_f1 / len(loader.dataset)


for epoch in range(1, 101):
    loss = train()
    acc = test(val_loader)
    print('Epoch: {:02d}, Loss: {:.4f}, Acc: {:.4f}'.format(epoch, loss, acc))


Epoch: 01, Loss: 0.8480, Acc: 0.5114
Epoch: 02, Loss: 0.5329, Acc: 0.5341
Epoch: 03, Loss: 0.4896, Acc: 0.5811
Epoch: 04, Loss: 0.4669, Acc: 0.6126
Epoch: 05, Loss: 0.4260, Acc: 0.6444
Epoch: 06, Loss: 0.3788, Acc: 0.7256
Epoch: 07, Loss: 0.3422, Acc: 0.7430
Epoch: 08, Loss: 0.3021, Acc: 0.7859
Epoch: 09, Loss: 0.2565, Acc: 0.8366
Epoch: 10, Loss: 0.2214, Acc: 0.8636
Epoch: 11, Loss: 0.1873, Acc: 0.8869
Epoch: 12, Loss: 0.1640, Acc: 0.9007
Epoch: 13, Loss: 0.1397, Acc: 0.9145
Epoch: 14, Loss: 0.1149, Acc: 0.9262
Epoch: 15, Loss: 0.1034, Acc: 0.9368
Epoch: 16, Loss: 0.0955, Acc: 0.9450
Epoch: 17, Loss: 0.0797, Acc: 0.9514
Epoch: 18, Loss: 0.0681, Acc: 0.9578
Epoch: 19, Loss: 0.0582, Acc: 0.9638
Epoch: 20, Loss: 0.0540, Acc: 0.9642
Epoch: 21, Loss: 0.0450, Acc: 0.9650
Epoch: 22, Loss: 0.0476, Acc: 0.9670
Epoch: 23, Loss: 0.0461, Acc: 0.9675
Epoch: 24, Loss: 0.0396, Acc: 0.9701
Epoch: 25, Loss: 0.0317, Acc: 0.9766
Epoch: 26, Loss: 0.0292, Acc: 0.9773
Epoch: 27, Loss: 0.0261, Acc: 0.9768
E

## geniepath

In [2]:
%%time
import argparse
import os.path as osp

import torch
from torch_geometric.datasets import PPI
from torch_geometric.data import DataLoader
from torch_geometric.nn import GATConv
from sklearn.metrics import f1_score

# parser = argparse.ArgumentParser()
# parser.add_argument('--model', type=str, default='GeniePath')
# args = parser.parse_args()
# assert args.model in ['GeniePath', 'GeniePathLazy']

path = osp.join('./', 'data', 'PPI')
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

dim = 256
lstm_hidden = 256
layer_num = 4


class Breadth(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Breadth, self).__init__()
        self.gatconv = GATConv(in_dim, out_dim, heads=1)#这里in_dim和out_dim都=dim=256
        # self.gatconv = GATConv(256, 256, heads=1)

    def forward(self, x, edge_index):
        x = torch.tanh(self.gatconv(x, edge_index))
        return x


class Depth(torch.nn.Module):
    def __init__(self, in_dim, hidden):
        super(Depth, self).__init__()
        self.lstm = torch.nn.LSTM(in_dim, hidden, 1, bias=False)

    def forward(self, x, h, c):
        x, (h, c) = self.lstm(x, (h, c))
        return x, (h, c)


class GeniePathLayer(torch.nn.Module):
    def __init__(self, in_dim):
        super(GeniePathLayer, self).__init__()
        self.breadth_func = Breadth(in_dim, dim)
        self.depth_func = Depth(dim, lstm_hidden)

    def forward(self, x, edge_index, h, c):
        x = self.breadth_func(x, edge_index)
        x = x[None, :]
        x, (h, c) = self.depth_func(x, h, c)
        x = x[0]
        return x, (h, c)


class GeniePath(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GeniePath, self).__init__()
        self.lin1 = torch.nn.Linear(in_dim, dim)
        self.gplayers = torch.nn.ModuleList(
            [GeniePathLayer(dim) for i in range(layer_num)])
        self.lin2 = torch.nn.Linear(dim, out_dim)

    def forward(self, x, edge_index):
        x = self.lin1(x)
        h = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        c = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        for i, l in enumerate(self.gplayers):
            x, (h, c) = self.gplayers[i](x, edge_index, h, c)
        x = self.lin2(x)
        return x


class GeniePathLazy(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super(GeniePathLazy, self).__init__()
        self.lin1 = torch.nn.Linear(in_dim, dim)
        self.breadths = torch.nn.ModuleList(
            [Breadth(dim, dim) for i in range(layer_num)])
        self.depths = torch.nn.ModuleList(
            [Depth(dim * 2, lstm_hidden) for i in range(layer_num)])
        self.lin2 = torch.nn.Linear(dim, out_dim)

    def forward(self, x, edge_index):
        x = self.lin1(x)
        h = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        c = torch.zeros(1, x.shape[0], lstm_hidden, device=x.device)
        h_tmps = []
        for i, l in enumerate(self.breadths):
            h_tmps.append(self.breadths[i](x, edge_index))
        x = x[None, :]
        for i, l in enumerate(self.depths):
            in_cat = torch.cat((h_tmps[i][None, :], x), -1)
            x, (h, c) = self.depths[i](in_cat, h, c)
        x = self.lin2(x[0])
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kwargs = {'GeniePath': GeniePath, 'GeniePathLazy': GeniePathLazy}
model = GeniePath(train_dataset.num_features,
                           train_dataset.num_classes).to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        num_graphs = data.num_graphs
        data.batch = None
        data = data.to(device)
        optimizer.zero_grad()
        hhh=model(data.x, data.edge_index)
        loss = loss_op(model(data.x, data.edge_index), data.y)
        total_loss += loss.item() * num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)


def test(loader):
    model.eval()

    ys, preds = [], []
    for data in loader:
        ys.append(data.y)
        with torch.no_grad():
            out = model(data.x.to(device), data.edge_index.to(device))
        preds.append((out > 0).float().cpu())

    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
    return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0

losslist=[]
for epoch in range(1, 101):
    loss = train()
    losslist.append(loss)
    val_f1 = test(val_loader)
    test_f1 = test(test_loader)
    print('Epoch: {:02d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}'.format(
        epoch, loss, val_f1, test_f1))
from matplotlib import pyplot as plt 
# %matplotlib inline

plt.plot(losslist)
plt.show()
sum([torch.numel(param) for param in model.parameters()])

Epoch: 01, Loss: 0.5796, Val: 0.3726, Test: 0.3727
Epoch: 02, Loss: 0.5505, Val: 0.4225, Test: 0.4226
Epoch: 03, Loss: 0.5436, Val: 0.4197, Test: 0.4198
Epoch: 04, Loss: 0.5390, Val: 0.4408, Test: 0.4422
Epoch: 05, Loss: 0.5395, Val: 0.4512, Test: 0.4516
Epoch: 06, Loss: 0.5347, Val: 0.4398, Test: 0.4411
Epoch: 07, Loss: 0.5354, Val: 0.4734, Test: 0.4750
Epoch: 08, Loss: 0.5328, Val: 0.4612, Test: 0.4622
Epoch: 09, Loss: 0.5287, Val: 0.5067, Test: 0.5108
Epoch: 10, Loss: 0.5286, Val: 0.4869, Test: 0.4887
Epoch: 11, Loss: 0.5252, Val: 0.4782, Test: 0.4800
Epoch: 12, Loss: 0.5222, Val: 0.5073, Test: 0.5099
Epoch: 13, Loss: 0.5256, Val: 0.4872, Test: 0.4897
Epoch: 14, Loss: 0.5218, Val: 0.4888, Test: 0.4923
Epoch: 15, Loss: 0.5209, Val: 0.4841, Test: 0.4873
Epoch: 16, Loss: 0.5202, Val: 0.4875, Test: 0.4926
Epoch: 17, Loss: 0.5213, Val: 0.4509, Test: 0.4559
Epoch: 18, Loss: 0.5230, Val: 0.4731, Test: 0.4776
Epoch: 19, Loss: 0.5178, Val: 0.5229, Test: 0.5252
Epoch: 20, Loss: 0.5172, Val: 0

<Figure size 640x480 with 1 Axes>

CPU times: user 54min 47s, sys: 4min 38s, total: 59min 25s
Wall time: 4min 17s


2406521

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
